In [1]:
import pandas as pd
import numpy as np
import joblib

from keras.models import load_model

from art.attacks.evasion import HopSkipJump, ZooAttack, BoundaryAttack
from art.attacks.evasion.fast_gradient import FastGradientMethod
from art.estimators.classification.query_efficient_bb import QueryEfficientGradientEstimationClassifier
from art.estimators.classification import KerasClassifier

from adversarialdefence.utils import ModelUtils, GeneralUtils

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/lib/python3.11/site-packages/art/estimators/certification/__init__.py:29: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


### ***Data Loading***

In [2]:
df = pd.read_csv('../csv/CICIDS2017_improved-preprocessed.csv')
df.drop(df.columns[0], axis=1, inplace=True)
df.shape

(1715326, 49)

In [3]:
df_benign = df[df['Label'] == 0]
df_anomalous = df[df['Label'] == 1]

print(f'Number of benign samples: {df_benign.shape[0]}')
print(f'Number of anomalous samples: {df_anomalous.shape[0]}')

Number of benign samples: 1432918
Number of anomalous samples: 282408


### ***Load models and scalers***

In [4]:
autoencoder = load_model('../modelli/autoencoder_best_weights_96-96.hdf5')
dnn = load_model('../modelli/DNN_best_weights_99.hdf5')

In [10]:
std_scaler_aut = joblib.load('../modelli/std_scaler_aut.bin')
std_scaler_dnn = joblib.load('../modelli/std_scaler_dnn.bin')
columns = df.copy().drop('Label', axis=1).columns 

/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
def checkAdversarialResults(X_advs, df_advs, attack_type, num_total_attempts, dnn_target, aut_binary_th=0.05, context=''):
    preds_advs_dnn = ModelUtils.binary_preds_supervised(dnn_target, X_advs)
    advs_samples_dnn = GeneralUtils.get_advs_samples(np.array(preds_advs_dnn), df_advs)
    print(f'Successful adversarial samples of type {attack_type} for DNN {context}: {advs_samples_dnn.shape[0]} / {num_total_attempts} - ASR: {(advs_samples_dnn.shape[0]/num_total_attempts) * 100}%')

    preds_advs_aut = ModelUtils.binary_preds_unsupervised(autoencoder, X_advs, aut_binary_th)
    advs_samples_aut = GeneralUtils.get_advs_samples(np.array(preds_advs_aut), df_advs)
    print(f'Successful adversarial samples of type {attack_type} for AUT {context}: {advs_samples_aut.shape[0]} / {num_total_attempts} - ASR: {(advs_samples_aut.shape[0]/num_total_attempts) * 100}%')

    preds_advs_dnn_by_aut = ModelUtils.binary_preds_unsupervised(autoencoder, advs_samples_dnn.copy().drop('Label', axis=1), aut_binary_th)
    common_advs_samples = GeneralUtils.get_advs_samples(np.array(preds_advs_dnn_by_aut), advs_samples_dnn)
    print(f'Successful adversarial samples of type {attack_type} for TARGET SYSTEM {context}: {common_advs_samples.shape[0]} / {num_total_attempts} - ATSR: {(common_advs_samples.shape[0]/num_total_attempts) * 100}%')

    #X_advs_with_preds = X_advs.assign(preds_dnn=preds_advs_dnn.astype(bool), preds_aut=np.array(preds_advs_aut).astype(bool))
    #X_advs_target_system = X_advs_with_preds[(X_advs_with_preds['preds_dnn'] == False) | (X_advs_with_preds['preds_aut'] == False)]   
    #print(f'Successful adversarial samples of type {attack_type} for TARGET SYSTEM {context}: {X_advs_target_system.shape[0]} / {X_advs.shape[0]} - Rate {(X_advs_target_system.shape[0] / X_advs.shape[0]) * 100}%')

### ***Perform Black Box attacks***

In [8]:
BASE_ADVS_CSV_PATH = '../csv/'

**HopSkipJump**

In [9]:
#tf.compat.v1.disable_eager_execution()
#dnn = load_model('../modelli/DNN_best_weights_99.hdf5')
X_anomalous_to_perturb = std_scaler_dnn.transform(df_anomalous.sample(n=50000).drop('Label', axis=1))
#dnn_target = KerasClassifier(model=dnn, clip_values=(X_anomalous_to_perturb.min(), X_anomalous_to_perturb.max()))

In [9]:
#hsj_attack = HopSkipJump(classifier=dnn_target)
#hsj_advs = hsj_attack.generate(X_anomalous_to_perturb)

In [29]:
df_hsj, X_hsj, y_hsj = GeneralUtils.get_data_with_advs(BASE_ADVS_CSV_PATH + 'hsj_new.csv')
checkAdversarialResults(X_hsj, df_hsj, 'HOPSKIPJUMP', X_anomalous_to_perturb.shape[0], dnn)

Successful adversarial samples of type HOPSKIPJUMP for DNN : 49667 / 50000 - ASR: 99.334%
Successful adversarial samples of type HOPSKIPJUMP for AUT : 21511 / 50000 - ASR: 43.022%
Successful adversarial samples of type HOPSKIPJUMP for TARGET SYSTEM : 21500 / 50000 - ATSR: 43.0%


**Boundary**

In [13]:
#tf.compat.v1.disable_eager_execution()
#dnn = load_model('../modelli/DNN_best_weights_99.hdf5')
X_anomalous_to_perturb = std_scaler_dnn.transform(df_anomalous.sample(n=50000).drop('Label', axis=1))
#dnn_target = KerasClassifier(model=dnn, clip_values=(X_anomalous_to_perturb.min(), X_anomalous_to_perturb.max()))

In [14]:
#boundary_attack = BoundaryAttack(estimator=dnn_target, targeted=False, max_iter=0, delta=0.001, epsilon=0.001, verbose=False)
#boundary_advs = boundary_attack.generate(X_anomalous_to_perturb)

In [25]:
df_boundary, X_boundary, y_boundary = GeneralUtils.get_data_with_advs(BASE_ADVS_CSV_PATH + 'boundary_new.csv')
checkAdversarialResults(X_boundary, df_boundary, 'BOUNDARY', X_anomalous_to_perturb.shape[0], dnn)

Successful adversarial samples of type BOUNDARY for DNN : 49847 / 50000 - ASR: 99.694%
Successful adversarial samples of type BOUNDARY for AUT : 191 / 50000 - ASR: 0.382%
Successful adversarial samples of type BOUNDARY for TARGET SYSTEM : 191 / 50000 - ATSR: 0.382%


**ZOO**

In [16]:
#tf.compat.v1.disable_eager_execution()
#dnn = load_model('../modelli/DNN_best_weights_99.hdf5')
X_anomalous_to_perturb = std_scaler_dnn.transform(df_anomalous.sample(n=50000).drop('Label', axis=1))
#dnn_target = KerasClassifier(model=dnn, clip_values=(X_anomalous_to_perturb.min(), X_anomalous_to_perturb.max()))

In [17]:
#zoo_attack = ZooAttack(dnn_target, confidence=0.5, targeted=False, learning_rate=0.2, nb_parallel=48, use_resize=False, use_importance=False)
#zoo_advs = zoo_attack.generate(X_anomalous_to_perturb)
#pd.DataFrame(zoo_advs, columns=columns).to_csv(BASE_ADVS_CSV_PATH + 'zoo_new.csv')

In [26]:
df_zoo, X_zoo, y_zoo = GeneralUtils.get_data_with_advs(BASE_ADVS_CSV_PATH + 'zoo_new.csv')
checkAdversarialResults(X_zoo, df_zoo, 'ZOO', X_anomalous_to_perturb.shape[0], dnn)

Successful adversarial samples of type ZOO for DNN : 3535 / 50000 - ASR: 7.07%
Successful adversarial samples of type ZOO for AUT : 40192 / 50000 - ASR: 80.384%
Successful adversarial samples of type ZOO for TARGET SYSTEM : 978 / 50000 - ATSR: 1.9560000000000002%


**Query Efficient**

In [19]:
#tf.compat.v1.disable_eager_execution()
#dnn = load_model('../modelli/DNN_best_weights_99.hdf5')
X_anomalous_to_perturb = std_scaler_dnn.transform(df_anomalous.sample(n=50000).drop('Label', axis=1))
#dnn_target = KerasClassifier(model=dnn, clip_values=(X_anomalous_to_perturb.min(), X_anomalous_to_perturb.max()))

In [20]:
#dnn_target_query_efficient_bb = QueryEfficientGradientEstimationClassifier(dnn_target, 20, 1 / 64.0, 0)
#fgsmBB_attack_dnn = FastGradientMethod(dnn_target_query_efficient_bb, eps=1)
#fgsmBB_advs_dnn = fgsmBB_attack_dnn.generate(X_anomalous_to_perturb, verbose=True)
#pd.DataFrame(fgsmBB_advs_dnn, columns=columns).to_csv(BASE_ADVS_CSV_PATH + 'query_eff_new.csv')

In [27]:
df_query_eff_new, X_query_eff_new, y_query_eff_new = GeneralUtils.get_data_with_advs(BASE_ADVS_CSV_PATH + 'query_eff_new.csv')
checkAdversarialResults(X_query_eff_new, df_query_eff_new, 'QUERY EFF', X_anomalous_to_perturb.shape[0], dnn)

Successful adversarial samples of type QUERY EFF for DNN : 24205 / 50000 - ASR: 48.41%
Successful adversarial samples of type QUERY EFF for AUT : 21389 / 50000 - ASR: 42.778%
Successful adversarial samples of type QUERY EFF for TARGET SYSTEM : 37 / 50000 - ATSR: 0.074%
